<a href="https://colab.research.google.com/github/tejpat98/Textbook-Summarisation/blob/main/Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Preprocessing textbook dataset:**
*   Download textbooks (OP / CC licences)
*   Convert PDF into raw text data
*   Split textbooks into chapters
*   Put into a labeled dataset format


Notes:
https://stackoverflow.com/questions/60754884/python-ocr-pytesseract-for-pdf 


In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
!apt-get install poppler-utils
!apt-get install tesseract-ocr
!pip install PyPDF2
!pip install pdf2image
!pip install pytesseract

In [ ]:
!sudo add-apt-repository ppa:alex-p/tesseract-ocr
!sudo apt-get update
!sudo apt install libtesseract-dev

In [ ]:
!nvidia-smi

In [ ]:
import GPUtil
GPUs = GPUtil.getGPUs()
for i, gpu in enumerate(GPUs):
  print('GPU {:d} ... Mem Free: {:.0f}MB / {:.0f}MB | Utilization {:3.0f}%'.format(i, gpu.memoryFree, gpu.memoryTotal, gpu.memoryUtil*100))


Convert PDF into Images

In [17]:
import os
from PIL import Image
from pdf2image import convert_from_path
import pytesseract

%cd '/content/drive/MyDrive/University/Computer Science - Level 3/FYP/Open and CC Textbooks (PDF)/27/'
path='/content/drive/MyDrive/University/Computer Science - Level 3/FYP/Open and CC Textbooks (PDF)/27/images/'
fileName = '27.pdf'
doc = convert_from_path(fileName, 300, output_folder=path, paths_only=True, fmt="jpeg")


/content/drive/MyDrive/University/Computer Science - Level 3/FYP/Open and CC Textbooks (PDF)/27


Convert Images into Text using PyTesseract

In [ ]:
for i in range(5,28):
  path = '/content/drive/MyDrive/University/Computer Science - Level 3/FYP/Open and CC Textbooks (PDF)/'+str(i)+'/images/'
  ImageNames = os.listdir("/content/drive/MyDrive/University/Computer Science - Level 3/FYP/Open and CC Textbooks (PDF)/"+str(i)+"/images/")
  print(len(ImageNames))
  PrefList= ImageNames[0].strip().split('-')[:-1]
  pref = '-'.join(PrefList)

  for i in range(1,len(ImageNames)):
    with open(path+'**output.txt', 'a') as f:
      txt = pytesseract.image_to_string(Image.open(path+str(pref)+'-'+str(i).zfill(len(str(len(ImageNames))))+'.jpg'))
      f.write("*****Page: " + str(i) +"*****")
      f.write("\n")
      f.write(txt)